In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas_datareader as pdr

plt.style.use('dark_background')
data = pd.read_csv('Asan.Pardakht.Pers.csv', index_col='<DTYYYYMMDD>')
data.index = pd.to_datetime(data.index, format='%Y%m%d', errors='ignore')
print(data.describe())
data = data[::-1]
close = data['<CLOSE>']
print("Going For Date",day)


            <FIRST>        <HIGH>         <LOW>       <CLOSE>       <VALUE>  \
count    726.000000    726.000000    726.000000    726.000000  7.260000e+02   
mean   16268.157025  16472.351240  15961.516529  16214.756198  2.769737e+10   
std     3577.939878   3628.376977   3517.282688   3572.827195  4.916397e+10   
min     9999.000000   9999.000000   9999.000000   9999.000000  1.301400e+09   
25%    14165.000000  14265.000000  13901.000000  14092.500000  8.091584e+09   
50%    15268.500000  15400.000000  15003.000000  15195.000000  1.468535e+10   
75%    17189.250000  17289.500000  16945.000000  17078.750000  3.050998e+10   
max    27500.000000  27600.000000  26987.000000  27014.000000  8.255174e+11   

              <VOL>     <OPENINT>        <OPEN>        <LAST>  
count  7.260000e+02    726.000000    726.000000    726.000000  
mean   1.752260e+06   1089.133609  16170.252066  16197.279614  
std    3.826796e+06   1475.642833   3613.125030   3555.437898  
min    9.864200e+04     75.00000

In [5]:

def macd(array):
    ema1 = array.ewm(span=12, adjust=False).mean()
    ema2 = array.ewm(span=26, adjust=False).mean()
    diff = ema2-ema1
    mac_ema = diff.ewm(span=9, adjust=False).mean()
    return diff, mac_ema

def getRsi(arr, ema_size=14):
    print("Getting RSI...")
    arr = arr.diff()
    arr = arr[1:]
    up, down = arr.copy(), arr.copy()
    up[up<0] = 0
    down[down>0] = 0
    rollup = up.ewm(span=ema_size).mean()
    rolldown = np.abs(down).ewm(span=ema_size).mean()
    rs = rollup/rolldown
    rsi = 100 - (100/(1+rs))
#     print(up.size, up)
#     print(arr.size)
#     print(rollup.size, rollup)
    print("RSI Calculated")
    return rsi
#     print(rsi)

def slice_data(data,currentDay, period='10 day'):
    return data[pd.Timestamp(currentDay)-pd.Timedelta(period):]

def slice_data_month(data,currentDay, period=3):
    return data[pd.Timestamp(currentDay)-pd.DateOffset(months=period):currentDay]

def checkCF(data, day, share):
    today = data[day]
    lastDay = data[:day][-2]
    data = data[:pd.Timestamp(day)-pd.Timedelta('1 day')]
    print("Cheking from:",data.index.max(),"To", data.index.min())
    max_ = data.max()
    min_ = data.min()
    print("Value:",lastDay ,"Max:", max_, "Min:",min_,"Difference:", max_-min_)
    if lastDay>max_:
        return 0
    elif lastDay < min_:
        return share
    sh = share / (max_-min_)
    value = share - sh*(lastDay-min_) 
    print("Share =>", value)
    return value
    space = np.linspace(share, 0, int(max_ - min_)+1)
    print("Space Created With:", data.size,"Days")
    return space[int(lastDay-min_)]

def checkRsi(data, day, ceil, floor, share):
    lastDay = data[:day][-2]
    print("Checking RSI Between",ceil, ",",floor,"Maximum Share:", share)
    print("Value is:", lastDay, "For Date", day, "Last Day:",data[:day].index[-2])
    if lastDay>ceil:
        return 0.0
    elif lastDay<floor:
        return share
    sh = share/(ceil-floor)
    value = share - sh*(lastDay - floor)
    print("Share =>",value,"%")
    return value

def checkTV(data, day, share):
    print("Checking Transaction Volume")
    min_ = data[:pd.Timestamp(day)-pd.Timedelta('1 day')].min()
    mean = data[:pd.Timestamp(day)-pd.Timedelta('1 day')].mean()
    lastDay = data[:day][-2]
    print("Minimum Transaction Specified Period:", min_)
    print("Mean Of Transactions:", mean)
    print("Checking", lastDay)
    if lastDay > mean:
        return share
    elif lastDay < min_:
        return 0.0;
    sh = share/(mean - min_)
    value = (lastDay - min_)*sh
    print("Value =>", value)
    return value

def getMacd(array):
    ema1 = array.ewm(span=12, adjust=False).mean()
    ema2 = array.ewm(span=26, adjust=False).mean()
    macd = ema1-ema2
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def emaDiff(data, long=26, short=12):
    short = data.ewm(span=short, adjust=False).mean()
    long = data.ewm(span=long, adjust=False).mean()
    diff = short - long
    print("Difference Calculated")
    diff[diff<0] = 0
    return diff

def checkEmaDiff(data, currentDay, share=10):
    data = data[:currentDay][-6:-1]
    diff = data.diff(1)
    print(data)
    print(diff)
    value = 0
    for i in range(len(data)-1,-1, -1):
        if diff[i] < 0:
            return value
        elif diff[i] > diff[i-1]:
            value += 2.5
        else:
            return value
            
#     print(data)
    return value

def checkMacd(macd, signal, day, share):
    hist = macd - signal
    hist = hist[:day]
    today = hist[-1]
    last = hist[-2]
    secondLast = hist[-3]
    print("Checking for two days befor" , day)
    print("Last Day Difference between MACD and it's Signal",last)
    if last > 0:
        return share
    return 0
    

In [6]:
# print("Checking ", day, data[day])
total = pd.Series(np.full(5, fill_value=0, dtype='float32'), np.array(['CF', 'RSI', 'TV', 'EMA Diff', 'MACD']))
print(total)

print("Checking CF:",day)
cf = np.full(4, fill_value=0, dtype='float32')
data_3_month = slice_data_month(close,day, 3)
print("Checking 3 Month")
cf[0] = checkCF(data_3_month, day, 10)

data_6_month = slice_data_month(close,day, 6)
print("Checking 6 Month")
cf[1] = checkCF(data_6_month, day, 10)

data_1_year = slice_data(close,day, '1Y')
print("Checking 1 Year")
cf[2] = checkCF(data_1_year, day, 10)

data_2_year = slice_data(close,day, '2Y')
print("Checking 2 Years")
cf[3] = checkCF(data_2_year, day, 10)

results = cf.sum()
total['CF'] = results
print("Value of CF", results)
print("\n###################")
print(total)

print("Checking RSI:",day)
rsi = getRsi(close)
print("RSI is:", rsi[pd.Timestamp(day)-pd.Timedelta('8d'):pd.Timestamp(day)+pd.Timedelta('2d')])
rsiResults = checkRsi(rsi, day, ceil=80, floor=20, share=20)
print("\nValue of RSI: ",rsiResults)
total['RSI'] = rsiResults
print("##################")
print(total)

print("Checking TRANSACTION VOLUME:",day)
volume = data['<VOL>']
volume_3_month = slice_data_month(volume,day, 1)
print("Last Month Data: ",volume_3_month)
tv = checkTV(volume_3_month, day, 20)
print("\nValue Of TV:",tv)
total['TV'] = tv
print("###########################")
print(total)

print("Checking EMA DIFF:",day)
emaDiffResult = checkEmaDiff(emaDiff(close), day, 10)
print("Result of EMAs Diff: ", emaDiffResult)
total['EMA Diff'] = emaDiffResult
print("###########################")
print(total)

print("Checking MACD:",day)
macd, signal = getMacd(close)
macdResults = checkMacd(macd, signal, day, 10)
print("Results:=>",macdResults)
total['MACD'] = macdResults
print("###########################")
print(total)


print("###########################")
print("###########################")
print("Date:",day, "\nSUM:" , total.sum().round(2),"%")

CF          0.0
RSI         0.0
TV          0.0
EMA Diff    0.0
MACD        0.0
dtype: float32
Checking CF: 2019-10-20
Checking 3 Month
Cheking from: 2019-10-16 00:00:00 To 2019-07-20 00:00:00
Value: 13623.0 Max: 17157.0 Min: 13216.0 Difference: 3941.0
Share => 8.967267191068256
Checking 6 Month
Cheking from: 2019-10-16 00:00:00 To 2019-04-20 00:00:00
Value: 13623.0 Max: 17157.0 Min: 13001.0 Difference: 4156.0
Share => 8.503368623676613
Checking 1 Year
Cheking from: 2019-10-16 00:00:00 To 2018-10-20 00:00:00
Value: 13623.0 Max: 17157.0 Min: 10722.0 Difference: 6435.0
Share => 5.491841491841492
Checking 2 Years
Cheking from: 2019-10-16 00:00:00 To 2017-10-21 00:00:00
Value: 13623.0 Max: 18793.0 Min: 10722.0 Difference: 8071.0
Share => 6.405649857514558
Value of CF 29.368124

###################
CF          29.368124
RSI          0.000000
TV           0.000000
EMA Diff     0.000000
MACD         0.000000
dtype: float32
Checking RSI: 2019-10-20
Getting RSI...
RSI Calculated
RSI is: <DTYYYY

In [2]:
tries = 0
while tries<3:
    temp = input()
    if temp in data.index:
        day = temp
        print(data.loc[temp])
        print(type(data.loc[temp]))
        break
    tries += 1

2019-10-20
<TICKER>     Asan.Pardakht.Pers
<FIRST>                   13899
<HIGH>                    13899
<LOW>                     12942
<CLOSE>                   13117
<VALUE>              8740069686
<VOL>                    668857
<OPENINT>                   517
<PER>                         D
<OPEN>                    13623
<LAST>                    12942
Name: 2019-10-20 00:00:00, dtype: object
<class 'pandas.core.series.Series'>


In [41]:
data.loc[:'2018-03-20']

,<TICKER>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>
<DTYYYYMMDD>,,,,,,,,,,,
2016-10-17,Asan.Pardakht.Pers,9999.0,9999.0,9999.0,9999.0,825517440000,82560000,1384,D,1000.0,9999.0
2016-10-18,Asan.Pardakht.Pers,10498.0,10498.0,10498.0,10498.0,20517490662,1954419,419,D,9999.0,10498.0
2016-10-19,Asan.Pardakht.Pers,11022.0,11022.0,11022.0,10910.0,3470563272,314876,282,D,10498.0,11022.0
2016-10-22,Asan.Pardakht.Pers,11455.0,11455.0,11455.0,11455.0,70183776960,6126912,406,D,10910.0,11455.0
2016-10-23,Asan.Pardakht.Pers,12027.0,12027.0,12027.0,12027.0,162206140491,13486833,2465,D,11455.0,12027.0
2016-10-24,Asan.Pardakht.Pers,12628.0,12628.0,12628.0,12628.0,17624381852,1395659,1614,D,12027.0,12628.0
2016-10-25,Asan.Pardakht.Pers,13259.0,13259.0,13259.0,13259.0,5881175299,443561,448,D,12628.0,13259.0
2016-10-26,Asan.Pardakht.Pers,13921.0,13921.0,13921.0,13921.0,237843361541,17085221,13368,D,13259.0,13921.0
2016-10-29,Asan.Pardakht.Pers,14617.0,14617.0,14617.0,14617.0,27474464008,1879624,2017,D,13921.0,14617.0


In [37]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [38]:
df.loc[dates[0]]

A   -1.698381
B    1.303043
C    0.357290
D   -0.910140
Name: 2013-01-01 00:00:00, dtype: float64

In [23]:
data['2019-10']

,<TICKER>,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>
<DTYYYYMMDD>,,,,,,,,,,,
2019-10-01,Asan.Pardakht.Pers,16649.0,16670.0,15636.0,16099.0,30265799072,1879933,1252,D,16458.0,15650.0
2019-10-02,Asan.Pardakht.Pers,15640.0,15795.0,15470.0,15631.0,18576027553,1188411,953,D,16099.0,15605.0
2019-10-05,Asan.Pardakht.Pers,15751.0,16090.0,15600.0,15894.0,18870111057,1187272,696,D,15631.0,15651.0
2019-10-06,Asan.Pardakht.Pers,16000.0,16100.0,15600.0,15882.0,11387565925,717002,543,D,15894.0,15900.0
2019-10-07,Asan.Pardakht.Pers,15899.0,15900.0,15300.0,15643.0,15488165747,990083,671,D,15882.0,15500.0
2019-10-08,Asan.Pardakht.Pers,15740.0,16018.0,14861.0,15316.0,24109248637,1574103,805,D,15643.0,15000.0
2019-10-09,Asan.Pardakht.Pers,14601.0,15000.0,14551.0,14647.0,9528938291,650567,561,D,15316.0,14999.0
2019-10-12,Asan.Pardakht.Pers,15000.0,15379.0,15000.0,15251.0,10574600065,693390,551,D,14647.0,15280.0
2019-10-13,Asan.Pardakht.Pers,15890.0,15890.0,14572.0,15098.0,8562703586,567155,612,D,15251.0,14850.0


In [3]:
pd.__version__

'0.25.3'